# Imports

In [1]:
import pandas as pd
import re
import spacy
from spacy import displacy

## Import da base

In [19]:
base_ouvidoria = pd.read_json('database/processos_ouvidoria.json')
base_sequencial = pd.read_json('database/processos_sequencial.json')

In [20]:
base = pd.concat([base_ouvidoria, base_sequencial],ignore_index=True)

In [21]:
base = base.drop_duplicates(subset='numero')

base.head()

,numero,juizo,classe_CNJ,eletronico,comarca,instancia,estado,andamentos_qtd,assuntos,andamentos
0,1774826420128169984,Segunda Vara de Violência Doméstica e Familiar...,Inquérito Policial,False,Recife,1GRAU,PE,4,[Crimes Previstos no Estatuto do Idoso],"[{'complemento': 'Central de Inquéritos', 'dat..."
1,1736234020128169984,Segunda Vara de Violência Doméstica e Familiar...,Inquérito Policial,False,Recife,1GRAU,PE,22,[Crimes Previstos no Estatuto do Idoso],"[{'complemento': 'Central de Inquéritos', 'dat..."
2,1544027620098170112,Terceira Vara de Violência Doméstica e Familia...,Ação Penal - Procedimento Sumário,False,Recife,1GRAU,PE,26,"[Decorrente de Violência Doméstica, Violência ...","[{'complemento': '', 'data': '12/09/2018 10:23..."
3,856377720148169984,Terceira Vara de Violência Doméstica e Familia...,Inquérito Policial,False,Recife,1GRAU,PE,14,[Ameaça],"[{'complemento': 'Ministério Público', 'data':..."
4,208506820168170016,Terceira Vara de Violência Doméstica e Familia...,Inquérito Policial,False,Recife,1GRAU,PE,9,"[Violência Doméstica Contra a Mulher, Ameaça]","[{'complemento': '', 'data': '20/06/2019 13:21..."


In [5]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13300 entries, 0 to 19216
Data columns (total 10 columns):
numero            13300 non-null int64
juizo             13300 non-null object
classe_CNJ        13300 non-null object
eletronico        13300 non-null bool
comarca           13300 non-null object
instancia         13300 non-null object
estado            13300 non-null object
andamentos_qtd    13300 non-null int64
assuntos          13300 non-null object
andamentos        13300 non-null object
dtypes: bool(1), int64(2), object(7)
memory usage: 1.0+ MB


## Leitura dos andamentos

## OBS

- Não podemos remover a data!! precisamos dela para dizer quanto tempo durou o processo

In [6]:
def remove_region(ent, text):
    old = text[ent.start_char:ent.end_char]
    new = []

    for word in list(old):
        if word != ' ':
            new.append('x')
        else:
            new.append(' ')

    new = ''.join(new)

    return text.replace(old, new)\

In [7]:
len(base['andamentos'])

13300

In [33]:
matchs = ['\d{1,2}\/\d{1,2}\/\d{2,4}', '\d{7}-\d{2}\.\d{4}\.\d{1}\.\d{2}\.\d{4}',
          '\d{1,2}\.\d{1,2}\.\d{2,4}', '\d+ de \w+ de \d+', '\d{2,4}:\d{2,4}:\d{2,4}']
my_model = spacy.load('pt')


for i, andamentos in enumerate(base['andamentos'][0:2]):
    new_andamentos = []
    
    for andamento in andamentos:
        andamento_aux = {}
        
        for key in andamento.keys():
            text = andamento[key]
            
            if key != 'data':                
                document = my_model(text)

                for ent in document.ents:
                    if ent.label_ == 'PER' and ent.text.isupper():
                        text = remove_region(ent, text)
                
                for regex in matchs:
                    text = re.sub(regex, 'xxxxxxxx', text)
                    
            andamento_aux[key] = text
        
        new_andamentos.append(andamento_aux)
    
    base['andamentos'][i] = new_andamentos
    new_andamentos = []

/home/mufasa/.pyenv/versions/3.7.3/envs/pyne/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
base['andamentos'][1]

[{'complemento': 'Central de Inquéritos',
  'data': '18/09/2014 14:34:00',
  'texto': '',
  'titulo': 'Autos entregues em carga ao Central de Inquéritos'},
 {'complemento': '',
  'data': '04/09/2014 10:19:00',
  'texto': 'Processo 0xxxxxxxx\r\nAcusado: Calos Alberto de Lima Barbosa e Fernando Lúcio de Lima Barbosa\r\n\r\n           \r\n           \r\n           \r\n           DESPACHO\r\n           \r\n           \r\n           \r\n           Em parecer psicológico de fl. 94/95 entendeu-se que fatos narrados no presente processo não tratam de situação relativa a violência ou opressão de gênero e, portanto, fogem da competência deste Juízo. Diante disto a Representante da Promotoria de Justiça requereu a remessa dos autos à Central de Inquéritos do Ministério Público (fl. 97-v).\r\n           Desta feita declino da competência e defiro o pedido do Ministério Público de fls. 18 para, de conseguinte, determinar a remessa destes autos à Central de Inquéritos do Ministério Público, após as 